In [1]:
import pandas as pd
import csv
import numpy as np
from pprint import pprint

import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn import linear_model
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import GridSearchCV, train_test_split

df_2012_train = pd.read_csv('df_2012_train.csv') 
df_2012_train = df_2012_train.set_index('fips')
df_2012_test = pd.read_csv('df_2012_test.csv') 
df_2012_test = df_2012_test.set_index('fips')
df_2016_train = pd.read_csv('df_2016_train.csv') 
df_2016_train = df_2016_train.set_index('county_name')
df_2016_test = pd.read_csv('df_2016_test.csv') 
df_2016_test = df_2016_test.set_index('county_name')

In [2]:
df_2012_train.head(2)

,state_abbreviation,county_name,diff_2012,land_area,population,perc_female,perc_u5,perc_o18,perc_o65,perc_2races,...,hc_value_1k,food_value_1k,median_income,per_cap_income,perc_pov,num_establishments,annual_payroll,vet_num_firms,fem_num_firms,min_num_firms
fips,,,,,,,,,,,,,,,,,,,,,
50011,VT,Franklin County,0.235485,0.607936,0.554590,0.852168,0.752126,0.454698,0.679059,0.334973,...,0.618064,0.598671,0.573469,0.538418,0.619473,0.426309,0.575188,0.366486,0.414746,0.166732
46093,SD,Meade County,-0.428453,0.800869,0.501383,0.814002,0.806294,0.452165,0.683144,0.451818,...,0.146813,0.044439,0.491513,0.502540,0.659653,0.397049,0.523883,0.330485,0.352493,0.177449


In [3]:
df_2012_train_x = df_2012_train.drop(columns = ['state_abbreviation', 'county_name', 'diff_2012'])
df_2012_train_y = df_2012_train[['diff_2012']]

df_2016_train_x = df_2016_train.drop(columns = ['state_abbreviation', 'fips', 'diff_2016'])
df_2016_train_y = df_2016_train[['diff_2016']]

df_2012_test_x = df_2012_test.drop(columns = ['state_abbreviation', 'county_name', 'diff_2012'])
df_2012_test_y = df_2012_test[['diff_2012']]

df_2016_test_x = df_2016_test.drop(columns = ['state_abbreviation', 'fips', 'diff_2016'])
df_2016_test_y = df_2016_test[['diff_2016']]

full_train_x = [df_2012_train_x, df_2016_train_x]
full_train_x = pd.concat(full_train_x)
df_2012_train_y.rename(columns={'diff_2012':'diff'}, inplace=True)
df_2016_train_y.rename(columns={'diff_2016':'diff'}, inplace=True)

full_train_y = [df_2012_train_y, df_2016_train_y]
full_train_y = pd.concat(full_train_y)

full_test_x = [df_2012_test_x, df_2016_test_x]
full_test_x = pd.concat(full_test_x)
df_2012_test_y.rename(columns={'diff_2012':'diff'}, inplace=True)
df_2016_test_y.rename(columns={'diff_2016':'diff'}, inplace=True)

full_test_y = [df_2012_test_y, df_2016_test_y]
full_test_y = pd.concat(full_test_y)

C:\Users\sajor\Anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [4]:
#2012 FEATURE SELECTION

from sklearn.feature_selection import RFE, VarianceThreshold, SelectFromModel

estimator = RidgeCV()

featureSelection = SelectFromModel(estimator, prefit=False, threshold='mean', max_features=None)
featureSelection.fit(df_2012_train_x, df_2012_train_y)

selectedFeatures = featureSelection.transform(df_2012_train_x)
#selectedFeatures

fs = df_2012_train_x.columns[featureSelection.get_support()]
#remaining features
keep = []
for i in range(len(fs)):
    keep.append(fs[i])
print(keep)

df_2012_train_x = df_2012_train_x[keep]
df_2012_test_x = df_2012_test_x[keep]

fts_2012 = keep




['population', 'perc_female', 'perc_o18', 'perc_o65', 'perc_white_not_hisp', 'median_oo_value', 'monthly_costs_m', 'monthly_costs_nm', 'avg_hh_size', 'perc_1yr_res', 'perc_f_emp_2065', 'hc_value_1k', 'perc_pov', 'num_establishments']


In [5]:
#2016 feature selection

estimator = RidgeCV()

featureSelection = SelectFromModel(estimator, prefit=False, threshold='mean', max_features=None)
featureSelection.fit(df_2016_train_x, df_2016_train_y)

selectedFeatures = featureSelection.transform(df_2016_train_x)
#selectedFeatures

fs = df_2016_train_x.columns[featureSelection.get_support()]
#remaining features
keep = []
for i in range(len(fs)):
    keep.append(fs[i])
print(keep)

df_2016_train_x = df_2016_train_x[keep]
df_2016_test_x = df_2016_test_x[keep]

fts_2016 = keep

['population', 'perc_o18', 'perc_o65', 'perc_white', 'perc_hisp', 'perc_white_not_hisp', 'median_oo_value', 'monthly_costs_nm', 'avg_hh_size', 'perc_hs', 'perc_f_emp_2065', 'hc_value_1k', 'median_income', 'perc_pov', 'num_establishments', 'annual_payroll']


In [6]:
estimator = RidgeCV()

featureSelection = SelectFromModel(estimator, prefit=False, threshold='mean', max_features=None)
featureSelection.fit(full_train_x, full_train_y)

selectedFeatures = featureSelection.transform(full_train_x)
#selectedFeatures

fs = full_train_x.columns[featureSelection.get_support()]
#remaining features
keep = []
for i in range(len(fs)):
    keep.append(fs[i])
print(keep)

full_train_x = full_train_x[keep]
full_test_x = full_test_x[keep]

fts_full = keep

['population', 'perc_female', 'perc_o18', 'perc_o65', 'perc_white', 'perc_hisp', 'perc_white_not_hisp', 'median_oo_value', 'monthly_costs_nm', 'avg_hh_size', 'perc_1yr_res', 'perc_f_lang', 'perc_hs', 'perc_f_emp_2065', 'hc_value_1k', 'median_income', 'perc_pov', 'num_establishments']


In [8]:
ls = RidgeCV()
pprint(ls.get_params())

{'alphas': array([ 0.1,  1. , 10. ]),
 'cv': None,
 'fit_intercept': True,
 'gcv_mode': None,
 'normalize': False,
 'scoring': None,
 'store_cv_values': False}


In [11]:



# Create a based model
rd = RidgeCV()
rd.fit(df_2012_train_x, df_2012_train_y)


RidgeCV(alphas=array([ 0.1,  1. , 10. ]), cv=None, fit_intercept=True,
    gcv_mode=None, normalize=False, scoring=None, store_cv_values=False)

In [12]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    mse = mean_squared_error(test_labels, predictions)
    rmse = np.sqrt(mse)
    print("mse:", mse)
    print("rmse:",rmse)
    mae = mean_absolute_error(test_labels, predictions)
    print("mae:", mae)
    r2 = explained_variance_score(test_labels, predictions)
    print("r2:",r2)

In [13]:
df_2012_train_x2 = df_2012_train.drop(columns = ['state_abbreviation', 'county_name', 'diff_2012'])
df_2012_train_y2 = df_2012_train[['diff_2012']]

df_2016_train_x2 = df_2016_train.drop(columns = ['state_abbreviation', 'fips', 'diff_2016'])
df_2016_train_y2 = df_2016_train[['diff_2016']]

df_2012_test_x2 = df_2012_test.drop(columns = ['state_abbreviation', 'county_name', 'diff_2012'])
df_2012_test_y2 = df_2012_test[['diff_2012']]

df_2016_test_x2 = df_2016_test.drop(columns = ['state_abbreviation', 'fips', 'diff_2016'])
df_2016_test_y2 = df_2016_test[['diff_2016']]

full_train_x2 = [df_2012_train_x2, df_2016_train_x2]
full_train_x2 = pd.concat(full_train_x2)
df_2012_train_y2.rename(columns={'diff_2012':'diff'}, inplace=True)
df_2016_train_y2.rename(columns={'diff_2016':'diff'}, inplace=True)

full_train_y2 = [df_2012_train_y2, df_2016_train_y2]
full_train_y2 = pd.concat(full_train_y2)

full_test_x2 = [df_2012_test_x2, df_2016_test_x2]
full_test_x2 = pd.concat(full_test_x2)
df_2012_test_y2.rename(columns={'diff_2012':'diff'}, inplace=True)
df_2016_test_y2.rename(columns={'diff_2016':'diff'}, inplace=True)

full_test_y2 = [df_2012_test_y2, df_2016_test_y2]
full_test_y2 = pd.concat(full_test_y2)

C:\Users\sajor\Anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [14]:
test_x = df_2016_test_x2[fts_2012]
full_x = full_test_x2[fts_2012]



print('2012 GRID Search on 2012')
grid_accuracy = evaluate(rd, df_2012_test_x, df_2012_test_y)
print('2012 GRID Search on 2016')
grid_accuracy = evaluate(rd, test_x, df_2016_test_y2)
print('2012 GRID Search on Full')
grid_accuracy = evaluate(rd, full_x, full_test_y2)

2012 GRID Search on 2012
mse: 0.04348455215253807
rmse: 0.20852949947798288
mae: 0.15751686646731655
r2: 0.48421638209433415
2012 GRID Search on 2016
mse: 0.6906896303093482
rmse: 0.8310773912875673
mae: 0.800462660681197
r2: 0.37846089673281624
2012 GRID Search on Full
mse: 0.36708709123094313
rmse: 0.605877125522117
mae: 0.47898976357425677
r2: -1.2504406789992148


In [ ]:
grid_search.best_params_

In [17]:
#fts = ['population', 'perc_o18', 'perc_white', 'perc_black', 'perc_white_not_hisp', 'perc_oo', 'monthly_costs_m', 'monthly_costs_nm', 'num_nonemployers', 'man_value_1k', 'wholesale_value_1k', 'num_establishments', 'annual_payroll', 'min_num_firms']
list(zip(fts_2012,list(rd.coef_)))

[('population',
  array([ 1.33856424,  0.52297583,  1.03617878, -0.96279437, -1.70395333,
          0.49913727, -0.81399007,  2.17005039, -0.51142843,  0.76015821,
          0.66278247, -0.15726452,  0.9397314 , -1.09078965]))]

In [18]:


# Fit the grid search to the data
rd.fit(df_2016_train_x, df_2016_train_y)



RidgeCV(alphas=array([ 0.1,  1. , 10. ]), cv=None, fit_intercept=True,
    gcv_mode=None, normalize=False, scoring=None, store_cv_values=False)

In [ ]:
rd.best_params_

In [20]:
test_x = df_2012_test_x2[fts_2016]
full_x = full_test_x2[fts_2016]

print('2016 GRID Search on 2012')
grid_accuracy = evaluate(rd, test_x, df_2012_test_y2)
print('2016 GRID Search on 2016')
grid_accuracy = evaluate(rd, df_2016_test_x, df_2016_test_y)
print('2016 GRID Search on Full')
grid_accuracy = evaluate(rd, full_x, full_test_y2)

2016 GRID Search on 2012
mse: 0.06473773767301304
rmse: 0.25443611707659164
mae: 0.2102288181820127
r2: 0.5640365374299572
2016 GRID Search on 2016
mse: 0.024493289778926362
rmse: 0.15650332194214397
mae: 0.12311502741016525
r2: 0.7334189309186719
2016 GRID Search on Full
mse: 0.044615513725969703
rmse: 0.2112238474367175
mae: 0.16667192279608897
r2: 0.5812862920729542


In [21]:
list(zip(fts_2016,list(rd.coef_)))

[('population',
  array([ 0.33580561,  0.3923771 , -0.30465362, -0.13883733, -0.27059636,
         -2.39136247,  0.76965471,  0.70639137, -0.30065118,  0.42169974,
          0.29028162, -0.03600764, -0.50468906,  0.29016815, -0.65655651,
          0.62018752]))]

In [9]:
rd = RidgeCV()
rd.fit(full_train_x, full_train_y)
rd.get_params

<bound method BaseEstimator.get_params of RidgeCV(alphas=array([ 0.1,  1. , 10. ]), cv=None, fit_intercept=True,
    gcv_mode=None, normalize=False, scoring=None, store_cv_values=False)>

In [23]:
test_x = df_2012_test_x2[fts_full]
test2_x = df_2016_test_x2[fts_full]

print('FULL GRID Search on 2012')
grid_accuracy = evaluate(rd, test_x, df_2012_test_y2)
print('FULL GRID Search on 2016')
grid_accuracy = evaluate(rd, test2_x, df_2016_test_y2)
print('FULL GRID Search on Full')
grid_accuracy = evaluate(rd, full_test_x, full_test_y)

FULL GRID Search on 2012
mse: 0.03433533206441007
rmse: 0.18529795483061887
mae: 0.14798796269202408
r2: 0.5915522188178145
FULL GRID Search on 2016
mse: 0.02505125884005582
rmse: 0.15827589469042916
mae: 0.12353600353882928
r2: 0.7275159610289349
FULL GRID Search on Full
mse: 0.029693295452232944
rmse: 0.17231742643224726
mae: 0.13576198311542667
r2: 0.6727614982515244


In [24]:
list(zip(fts_full,list(rd.coef_)))

[('population',
  array([ 0.84251672,  0.34033067,  0.79165593, -0.80152614, -0.07050278,
         -0.46797119, -2.18372914,  0.63139935,  0.56266206, -0.33084884,
          0.46485121,  0.35798017,  0.44736378,  0.47557519, -0.0606614 ,
         -0.54039527,  0.30495891, -0.56546128]))]